In [3]:
import numpy as np
from itertools import product
import math
import random
import copy

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

In [8]:
class IsingModel:
    def __init__(self, data, lr=0.1):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(data)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # First calculate the correlation
        self.data_correlation = self.calculate_corr(self.data_activations_matrix)
        # Then, the mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
        
        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=.01, size=self.data_correlation.shape)
        lind = np.tril_indices_from(self.J)
        self.J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
        
    # Calculate the correlation of a matrix
    def calculate_corr(self, activations_matrix):
        num_spins = activations_matrix.shape[1]
        num_samples_data = activations_matrix.shape[0]
        
        corr_matrix = np.zeros((self.num_spins, self.num_spins))
        for sigma in activations_matrix:
            # Calculate \sigma_i * \sigma_j
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]
        corr_matrix = corr_matrix / float(self.num_samples_data)
        
        return corr_matrix
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            for j in range(i+1, self.num_spins):
                energy += sigma[i]*sigma[j]*self.J[i][j]

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
                
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            model_correlation = np.zeros(self.J.shape)
            model_mean = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                
                # Calculate the expectation of the correlations
                for i in range(0, self.num_spins):
                    for j in range(i+1, self.num_spins):
                        model_correlation[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]

                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    model_mean[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            # Take the step
            oldJ = copy.deepcopy(self.J)
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Calculate the variation of J for the termination condition
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
            
            epoch += 1

        print(self.H, 'H')
        print(self.J, 'J')
        
    def random_walk(self, max_steps):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            if delta_energy<=0: # If delta_energy<0, accept the spin flip, energetically favourable
                model_activations_list.append(sigma_t)
            

            else: # Check if transition probability p < uniform r
                p = math.exp(-delta_energy)
                r = random.random()

                if r<=p: # Accept spin flip
                    model_activations_list.append(sigma_t)
                else: # Refuse spin flip
                    model_activations_list.append(sigma_0)
            
            sigma_0 = sigma_t
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300, random_walk_len = 100):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(random_walk_len)  
            model_activations_matrix = np.array(model_activations_list)
            # Calculate the data expectations
            model_correlation = self.calculate_corr(model_activations_matrix)
            model_mean = np.mean(model_activations_matrix, axis=0)
            
            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            
            oldJ = copy.deepcopy(self.J)
            # Take the step
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            
            oldH = copy.deepcopy(self.H)
            # Take the step
            self.H = self.H + stepH
            
            # Calculate the variation of J and H for the termination condition
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%10 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
                                    
            epoch +=1

        print(self.H, 'H')
        print(self.J, 'J')
        

In [9]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]

In [10]:
ising = IsingModel(activations_list, lr=0.0005)

In [7]:
ising.train_metropolis(max_epochs = 15000, random_walk_len = 400)


Epoch 10 TotalParamVariation 0.007615
[[ 0.    5.    4.25  0.25]
 [ 0.    0.   -1.75 -0.75]
 [ 0.    0.    0.    2.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[-0.045  0.045 -0.06  -0.08 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 20 TotalParamVariation 0.0089075
[[ 0.   -0.75  7.   -3.5 ]
 [ 0.    0.   -1.75 -2.75]
 [ 0.    0.    0.   -2.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[-0.025  0.01  -0.015 -0.015] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 30 TotalParamVariation 0.007275
[[ 0.  -2.  -1.   1.5]
 [ 0.   0.   1.   4. ]
 [ 0.   0.   0.   3.5]
 [ 0.   0.   0.   0. ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[0.025 0.015 0.0

Epoch 270 TotalParamVariation 0.0051775
[[ 0.   -0.25 -4.   -2.  ]
 [ 0.    0.    1.75 -0.75]
 [ 0.    0.    0.    0.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.02   0.015 -0.02   0.09 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 280 TotalParamVariation 0.0093025
[[ 0.    0.   -0.75 -3.5 ]
 [ 0.    0.    3.75  1.5 ]
 [ 0.    0.    0.   -7.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.04   0.03   0.185 -0.05 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 290 TotalParamVariation 0.00763
[[ 0.   -1.75 -1.25 -2.  ]
 [ 0.    0.   -4.   -2.75]
 [ 0.    0.    0.    3.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Da

Epoch 510 TotalParamVariation 0.0081775
[[ 0.   -2.75 -1.25 -0.25]
 [ 0.    0.    5.    2.  ]
 [ 0.    0.    0.   -4.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[0.015 0.12  0.08  0.08 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 520 TotalParamVariation 0.0078075
[[ 0.    2.25 -2.   -0.5 ]
 [ 0.    0.   -2.75 -6.25]
 [ 0.    0.    0.    1.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[0.115 0.05  0.075 0.005] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 530 TotalParamVariation 0.0073425
[[ 0.    2.25 -3.   -3.5 ]
 [ 0.    0.    2.75 -1.25]
 [ 0.    0.    0.    0.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[-0

Epoch 750 TotalParamVariation 0.0073725
[[ 0.   -0.5  -6.5   1.25]
 [ 0.    0.   -1.   -1.25]
 [ 0.    0.    0.   -3.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[-0.23  -0.21   0.18  -0.155] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 760 TotalParamVariation 0.00521
[[ 0.   -0.5  -0.75 -3.25]
 [ 0.    0.   -3.75 -0.25]
 [ 0.    0.    0.   -2.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.015 -0.055  0.04   0.03 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 770 TotalParamVariation 0.0081075
[[ 0.   -3.5   3.5   2.75]
 [ 0.    0.    3.5   0.75]
 [ 0.    0.    0.    1.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Da

Epoch 990 TotalParamVariation 0.0110875
[[ 0.   -0.75  2.75  5.75]
 [ 0.    0.   -3.5  -1.5 ]
 [ 0.    0.    0.    8.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[-0.02  -0.025  0.105  0.035] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1000 TotalParamVariation 0.0048325
[[ 0.   -1.25 -2.25 -0.75]
 [ 0.    0.   -2.5  -2.5 ]
 [ 0.    0.    0.    0.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[-0.005 -0.13   0.05   0.09 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1010 TotalParamVariation 0.00821
[[ 0.   1.5 -2.5  3.5]
 [ 0.   0.   1.5  2.5]
 [ 0.   0.   0.   2.5]
 [ 0.   0.   0.   0. ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.105 -0.

...

Epoch 13940 TotalParamVariation 0.00547
[[ 0.  -1.   2.5  2.5]
 [ 0.   0.   0.  -1.5]
 [ 0.   0.   0.   3.5]
 [ 0.   0.   0.   0. ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.1  -0.26  0.28  0.06] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 13950 TotalParamVariation 0.00706
[[ 0.  -2.5  3.  -1.5]
 [ 0.   0.  -2.  -3.5]
 [ 0.   0.   0.   2.5]
 [ 0.   0.   0.   0. ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.275 -0.445  0.115 -0.015] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 13960 TotalParamVariation 0.0065575
[[ 0.    5.    4.   -0.25]
 [ 0.    0.    1.    1.75]
 [ 0.    0.    0.   -0.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28  -0.24   0.18  -0.005]

Epoch 14180 TotalParamVariation 0.00868
[[ 0.    1.    3.25  1.25]
 [ 0.    0.    1.25  3.25]
 [ 0.    0.    0.   -6.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.235 -0.105  0.33   0.03 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14190 TotalParamVariation 0.0063275
[[ 0.   -4.    5.   -1.75]
 [ 0.    0.    0.    1.25]
 [ 0.    0.    0.    0.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.26  -0.07   0.31  -0.025] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14200 TotalParamVariation 0.00705
[[ 0.    0.25  2.25  1.  ]
 [ 0.    0.   -0.5  -3.25]
 [ 0.    0.    0.    6.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Cor

Epoch 14420 TotalParamVariation 0.0088475
[[ 0.   -2.25  4.25 -0.75]
 [ 0.    0.    6.    1.5 ]
 [ 0.    0.    0.   -3.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.205 -0.11   0.28  -0.04 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14430 TotalParamVariation 0.0053625
[[ 0.   -4.75  0.75 -1.25]
 [ 0.    0.   -2.5   0.  ]
 [ 0.    0.    0.    2.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.325 -0.29   0.18   0.04 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14440 TotalParamVariation 0.006255
[[ 0.   -2.75  3.5   1.25]
 [ 0.    0.    0.25 -1.  ]
 [ 0.    0.    0.    3.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] 

Epoch 14660 TotalParamVariation 0.003995
[[ 0.   -0.25  0.25  3.  ]
 [ 0.    0.    0.5   1.75]
 [ 0.    0.    0.   -1.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.125 -0.1    0.31   0.025] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14670 TotalParamVariation 0.0063375
[[0.   0.   0.75 1.  ]
 [0.   0.   0.75 4.  ]
 [0.   0.   0.   4.75]
 [0.   0.   0.   0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.2   -0.25   0.205  0.08 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14680 TotalParamVariation 0.011525
[[ 0.    5.   -3.75  4.25]
 [ 0.    0.   -4.25 -1.75]
 [ 0.    0.    0.    3.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.14 

Epoch 14900 TotalParamVariation 0.00721
[[ 0.   2.  -1.5  0.5]
 [ 0.   0.   5.5 -3. ]
 [ 0.   0.   0.  -0.5]
 [ 0.   0.   0.   0. ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.145 -0.085  0.415 -0.065] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14910 TotalParamVariation 0.0120025
[[ 0.   -0.75 -6.   -6.  ]
 [ 0.    0.    5.75  4.75]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.09  -0.055  0.21  -0.11 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14920 TotalParamVariation 0.0068475
[[ 0.   -4.   -2.   -1.25]
 [ 0.    0.    1.5  -1.25]
 [ 0.    0.    0.   -3.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.155

In [17]:
ising.train(max_epochs = 15000)

NameError: name 'ising' is not defined

#### Only H

In [10]:
class SimpleIsingModel:
    def __init__(self, data, lr=0.0005):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(data)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # The mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            model_mean = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    model_mean[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            
            oldH = copy.deepcopy(self.H)
            # Take the step
            self.H = self.H + stepH
            
            # Calculate the variation of J for the termination condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))

            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_mean, 'Diff')
                print()
            
            epoch += 1

        print(self.data_mean, 'Spins average')
        print(self.H, 'H')
        
    def random_walk(self, max_steps):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        burn_in_len = 100
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            # If delta_energy<0, accept the spin flip, energetically favourable
            if delta_energy>0:  # Ohterwise, check if transition probability p < uniform r

                p = math.exp(-delta_energy)
                r = random.random()

                if r>p: # If transition probability is lower than r, refuse the update, otherwise accept
                    sigma_t = copy.deepcopy(sigma_0)
                    energy_t = energy_0
            
            # Save the state if the burn-in phase has passed
            if i > burn_in_len:
                model_activations_list.append(sigma_t)

            sigma_0 = copy.deepcopy(sigma_t)
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300, random_walk_len = 100):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0000005
        
        epoch = 1
        
        print('Epoch 0')
        print(self.H)
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(random_walk_len)   
            model_activations_matrix = np.array(model_activations_list)

            # Calculate the data expectations
            model_mean = np.mean(model_activations_matrix, axis=0)
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of H for the stop condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))
            
            if epoch%10 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_mean, 'Diff')
                print()

            
            epoch +=1
        
        print(self.data_mean, 'Spins average')
        print(self.H, 'H')


In [15]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
s_ising = SimpleIsingModel(activations_list, lr=0.005)


In [16]:
s_ising.train_metropolis( max_epochs = 15000, random_walk_len = 400)

Epoch 0
[-0.00145754 -0.00258427 -0.00546664  0.00997067]
Epoch 10 TotalParamVariation 0.00319398
[ 0.11705686 -0.10367893  0.1638796  -0.02341137] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.13294314 -0.14632107  0.3361204   0.02341137] Diff

Epoch 20 TotalParamVariation 0.00449833
[0.10367893 0.01003344 0.01003344 0.00334448] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.14632107 -0.26003344  0.48996656 -0.00334448] Diff

Epoch 30 TotalParamVariation 0.00510033
[-0.11036789  0.0367893   0.13043478  0.00334448] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.36036789 -0.2867893   0.36956522 -0.00334448] Diff

Epoch 40 TotalParamVariation 0.00379599
[-0.02341137 -0.19732441  0.10367893 -0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.27341137 -0.05267559  0.39632107  0.0367893 ] Diff

Epoch 50 TotalParamVariation 0.00446488
[ 0.10367893 -0.03010033  0.05016722  0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.146

Epoch 440 TotalParamVariation 0.00199833
[ 0.14381271 -0.31772575  0.29765886 -0.02341137] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[0.10618729 0.06772575 0.20234114 0.02341137] Diff

Epoch 450 TotalParamVariation 0.00173913
[ 0.34448161 -0.32441472  0.35785953 -0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.09448161  0.07441472  0.14214047  0.0367893 ] Diff

Epoch 460 TotalParamVariation 0.0025
[ 0.29765886 -0.11036789  0.22408027  0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.04765886 -0.13963211  0.27591973 -0.0367893 ] Diff

Epoch 470 TotalParamVariation 0.00233278
[ 0.090301   -0.27759197  0.29765886  0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.159699    0.02759197  0.20234114 -0.07692308] Diff

Epoch 480 TotalParamVariation 0.00301003
[ 0.37123746 -0.27759197  0.12374582  0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.12123746  0.02759197  0.37625418 -0.07692308] Diff

Epoch 49

Epoch 950 TotalParamVariation 0.00106187
[ 0.34448161 -0.2173913   0.45819398 -0.04347826] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.09448161 -0.0326087   0.04180602  0.04347826] Diff

Epoch 960 TotalParamVariation 0.00340301
[ 0.090301   -0.43143813  0.37123746  0.21070234] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.159699    0.18143813  0.12876254 -0.21070234] Diff

Epoch 970 TotalParamVariation 0.00156355
[ 0.15719064 -0.33110368  0.47826087 -0.11705686] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[0.09280936 0.08110368 0.02173913 0.11705686] Diff

Epoch 980 TotalParamVariation 0.00269231
[ 0.17725753  0.07023411  0.44481605 -0.090301  ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.07274247 -0.32023411  0.05518395  0.090301  ] Diff

Epoch 990 TotalParamVariation 0.00176421
[ 0.19063545 -0.31103679  0.33110368  0.06354515] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.05936455  0.06103679  0.16889632 -0.06354515] Diff

Epoc

Epoch 1410 TotalParamVariation 0.00074415
[ 0.26421405 -0.23076923  0.57859532  0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.01421405 -0.01923077 -0.07859532 -0.0367893 ] Diff

Epoch 1420 TotalParamVariation 0.00158027
[ 0.31772575 -0.12374582  0.5451505  -0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.06772575 -0.12625418 -0.0451505   0.07692308] Diff

Epoch 1430 TotalParamVariation 0.00230769
[ 0.36454849 -0.36454849  0.30434783 -0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.11454849  0.11454849  0.19565217  0.0367893 ] Diff

Epoch 1440 TotalParamVariation 0.00199833
[ 0.07023411 -0.31103679  0.45150502 -0.11036789] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[0.17976589 0.06103679 0.04849498 0.11036789] Diff

Epoch 1450 TotalParamVariation 0.00250836
[ 0.33779264 -0.45150502  0.38461538  0.09698997] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.08779264  0.20150502  0.11538462 -0.09698997] Diff


[-0.01421405  0.12123746  0.16889632  0.090301  ] Diff

Epoch 1830 TotalParamVariation 0.00188963
[ 0.35117057 -0.26421405  0.61204013  0.15050167] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.10117057  0.01421405 -0.11204013 -0.15050167] Diff

Epoch 1840 TotalParamVariation 0.0014214
[ 0.05016722 -0.24414716  0.4916388  -0.07023411] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.19983278 -0.00585284  0.0083612   0.07023411] Diff

Epoch 1850 TotalParamVariation 0.00168896
[ 0.13043478 -0.13043478  0.43812709  0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.11956522 -0.11956522  0.06187291 -0.0367893 ] Diff

Epoch 1860 TotalParamVariation 0.00190635
[ 0.31772575 -0.39130435  0.44481605  0.11705686] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.06772575  0.14130435  0.05518395 -0.11705686] Diff

Epoch 1870 TotalParamVariation 0.00107023
[ 0.37123746 -0.25752508  0.45150502  0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average D

...

Epoch 12780 TotalParamVariation 0.00168896
[ 0.23745819 -0.1638796   0.29765886  0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01254181 -0.0861204   0.20234114 -0.0367893 ] Diff

Epoch 12790 TotalParamVariation 0.00272575
[0.07692308 0.02341137 0.43143813 0.03010033] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.17307692 -0.27341137  0.06856187 -0.03010033] Diff

Epoch 12800 TotalParamVariation 0.00221572
[ 0.2173913  -0.31772575  0.69899666 -0.14381271] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.0326087   0.06772575 -0.19899666  0.14381271] Diff

Epoch 12810 TotalParamVariation 0.00093645
[ 0.25083612 -0.33779264  0.4180602   0.01672241] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.00083612  0.08779264  0.0819398  -0.01672241] Diff

Epoch 12820 TotalParamVariation 0.00280936
[ 0.41137124 -0.31103679  0.35117057  0.19063545] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.16137124  0.06103679  0.14882943 -0.19063545] 

Epoch 13310 TotalParamVariation 0.00187291
[ 0.23745819 -0.03010033  0.60535117  0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01254181 -0.21989967 -0.10535117 -0.0367893 ] Diff

Epoch 13320 TotalParamVariation 0.00158863
[ 0.46488294 -0.27090301  0.52508361  0.05685619] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.21488294  0.02090301 -0.02508361 -0.05685619] Diff

Epoch 13330 TotalParamVariation 0.00068562
[ 0.23076923 -0.2173913   0.43143813  0.01672241] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01923077 -0.0326087   0.06856187 -0.01672241] Diff

Epoch 13340 TotalParamVariation 0.00103679
[ 0.23076923 -0.20401338  0.51839465  0.12374582] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01923077 -0.04598662 -0.01839465 -0.12374582] Diff

Epoch 13350 TotalParamVariation 0.00065217
[ 0.27759197 -0.25752508  0.57859532  0.01672241] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.02759197  0.00752508 -0.07859532 -0.016722

Epoch 13770 TotalParamVariation 0.00319398
[ 0.42474916 -0.41137124  0.69230769 -0.11036789] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.17474916  0.16137124 -0.19230769  0.11036789] Diff

Epoch 13780 TotalParamVariation 0.00197324
[ 0.24414716 -0.15719064  0.79264214  0.00334448] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00585284 -0.09280936 -0.29264214 -0.00334448] Diff

Epoch 13790 TotalParamVariation 0.00210702
[ 0.47826087 -0.33110368  0.43812709 -0.05016722] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.22826087  0.08110368  0.06187291  0.05016722] Diff

Epoch 13800 TotalParamVariation 0.00150502
[ 0.12374582 -0.19732441  0.6187291  -0.00334448] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.12625418 -0.05267559 -0.1187291   0.00334448] Diff

Epoch 13810 TotalParamVariation 0.002199
[ 0.51170569 -0.15719064  0.4916388   0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.26170569 -0.09280936  0.0083612  -0.07692308

Epoch 14200 TotalParamVariation 0.00105351
[ 0.32441472 -0.27759197  0.55183946  0.05685619] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.07441472  0.02759197 -0.05183946 -0.05685619] Diff

Epoch 14210 TotalParamVariation 0.00149666
[ 0.34448161 -0.19063545  0.43143813 -0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.09448161 -0.05936455  0.06856187  0.07692308] Diff

Epoch 14220 TotalParamVariation 0.00122074
[ 0.30434783 -0.27759197  0.57859532  0.08361204] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.05434783  0.02759197 -0.07859532 -0.08361204] Diff

Epoch 14230 TotalParamVariation 0.00197324
[ 0.23076923 -0.13043478  0.57859532 -0.17725753] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01923077 -0.11956522 -0.07859532  0.17725753] Diff

Epoch 14240 TotalParamVariation 0.00188963
[ 0.11705686 -0.17725753  0.4180602   0.090301  ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.13294314 -0.07274247  0.0819398  -0.090301

Epoch 14690 TotalParamVariation 0.00179766
[ 0.32441472 -0.15050167  0.45819398 -0.14381271] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.07441472 -0.09949833  0.04180602  0.14381271] Diff

Epoch 14700 TotalParamVariation 0.00172241
[ 0.35785953 -0.44481605  0.53846154  0.00334448] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.10785953  0.19481605 -0.03846154 -0.00334448] Diff

Epoch 14710 TotalParamVariation 0.00211538
[ 0.090301   -0.34448161  0.63879599 -0.03010033] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.159699    0.09448161 -0.13879599  0.03010033] Diff

Epoch 14720 TotalParamVariation 0.00142977
[ 0.11705686 -0.31103679  0.4180602   0.01003344] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.13294314  0.06103679  0.0819398  -0.01003344] Diff

Epoch 14730 TotalParamVariation 0.0010786
[ 0.27090301 -0.18394649  0.55183946  0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.02090301 -0.06605351 -0.05183946 -0.0769230

In [17]:
s_ising.train( max_epochs = 10000)

Epoch 100 TotalParamVariation 0.00047512
[ 0.00233335 -0.01885997  0.03009478 -0.00152759] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.24766665 -0.23114003  0.46990522  0.00152759] Diff

Epoch 200 TotalParamVariation 0.00045197
[ 0.01441416 -0.03012881  0.05297817 -0.00145307] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.23558584 -0.21987119  0.44702183  0.00145307] Diff

Epoch 300 TotalParamVariation 0.00042997
[ 0.02590192 -0.04084136  0.07469721 -0.00138219] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.22409808 -0.20915864  0.42530279  0.00138219] Diff

Epoch 400 TotalParamVariation 0.00040909
[ 0.03682335 -0.05102351  0.09529727 -0.00131476] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.21317665 -0.19897649  0.40470273  0.00131476] Diff

Epoch 500 TotalParamVariation 0.00038926
[ 0.0472046  -0.06070041  0.11482608 -0.00125063] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.2027954  -0.18929959  0.38517392  0.00125063] Diff



Epoch 5000 TotalParamVariation 4.896e-05
[ 2.27103676e-01 -2.28554964e-01  4.46550616e-01 -1.31740968e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.02289632 -0.02144504  0.05344938  0.00013174] Diff

Epoch 5100 TotalParamVariation 4.688e-05
[ 2.28164085e-01 -2.29547497e-01  4.48645905e-01 -1.25314318e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.02183592 -0.0204525   0.0513541   0.00012531] Diff

Epoch 5200 TotalParamVariation 4.49e-05
[ 2.29174890e-01 -2.30493658e-01  4.50654513e-01 -1.19201176e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.02082511 -0.01950634  0.04934549  0.0001192 ] Diff

Epoch 5300 TotalParamVariation 4.3e-05
[ 2.30138455e-01 -2.31395652e-01  4.52580353e-01 -1.13386249e-04] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01986154 -0.01860435  0.04741965  0.00011339] Diff

Epoch 5400 TotalParamVariation 4.118e-05
[ 2.31057026e-01 -2.32255576e-01  4.54427141e-01 -1.07854987e-04] Average Model
[ 0.25 -0.25  0.5 

Epoch 9500 TotalParamVariation 7.35e-06
[ 2.47252470e-01 -2.47424974e-01  4.90642771e-01 -1.38775850e-05] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 2.74753031e-03 -2.57502553e-03  9.35722924e-03  1.38775850e-05] Diff

Epoch 9600 TotalParamVariation 7.05e-06
[ 2.47378492e-01 -2.47543074e-01  4.90991333e-01 -1.32006022e-05] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 2.62150787e-03 -2.45692571e-03  9.00866728e-03  1.32006022e-05] Diff

Epoch 9700 TotalParamVariation 6.77e-06
[ 2.47498727e-01 -2.47655751e-01  4.91326765e-01 -1.25566442e-05] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 2.50127340e-03 -2.34424907e-03  8.67323512e-03  1.25566442e-05] Diff

Epoch 9800 TotalParamVariation 6.49e-06
[ 2.47613440e-01 -2.47763254e-01  4.91649572e-01 -1.19441001e-05] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 2.38656037e-03 -2.23674597e-03  8.35042763e-03  1.19441001e-05] Diff

Epoch 9900 TotalParamVariation 6.23e-06
[ 2.47722885e-01 -2.47865822e-01  4.